# DeepLabCut Toolbox - DEMO for maDLC
https://github.com/AlexEMG/DeepLabCut

#### The notebook accompanies the following preprint:

*Multi-animal pose estimation and tracking with DeepLabCut.* A. Mathis et al, 2020 in prep.

This notebook illustrates how to:

- create a multi-animal project
- label the data 
- check the labels 
- create a multi-animal training dataset
- train your new deep neural network 

### You must have deeplabcut 2.2 installed (upgrade here if needed):

In [ ]:
!pip install --upgrade deeplabcut

## Import the toolbox:

In [1]:
%cd '/home/mackenzie/DEEPLABCUT/DLCdev-DLC2.2/'

/home/mackenzie/DEEPLABCUT/DLCdev-DLC2.2


In [3]:
import deeplabcut
deeplabcut.__version__

'2.2b0'

# Create a new project

It is always good idea to keep the projects seperate if you want to use different networks to analze your data. You should use one project if you are tracking similar subjects/items even if in different environments. This function creates a new project with sub-directories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

In [4]:
task='penguins5' # Enter the name of your experiment Task
experimenter='teamDLC' # Enter the name of the experimenter
video=['/home/mackenzie/Desktop/penguindemo.mp4'] # Enter the paths of your videos OR FOLDER you want to grab frames from.
#CRUCIAL: you should take as diverse frames as possible from multiple videos and animals!

deeplabcut.create_new_project(task,experimenter,video,copy_videos=True, multianimal=True) 

# NOTE: The function returns the path, where your project is. 
# You could also enter this manually (e.g. if the project is already created and you want to pick up, where you stopped...)
#path_config_file = '/home/Mackenzie/myDLCProjectFolders/config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

Created "/home/mackenzie/DEEPLABCUT/DLCdev-DLC2.2/penguins5-teamDLC-2020-03-02/videos"
Created "/home/mackenzie/DEEPLABCUT/DLCdev-DLC2.2/penguins5-teamDLC-2020-03-02/labeled-data"
Created "/home/mackenzie/DEEPLABCUT/DLCdev-DLC2.2/penguins5-teamDLC-2020-03-02/training-datasets"
Created "/home/mackenzie/DEEPLABCUT/DLCdev-DLC2.2/penguins5-teamDLC-2020-03-02/dlc-models"
Copying the videos
/home/mackenzie/DEEPLABCUT/DLCdev-DLC2.2/penguins5-teamDLC-2020-03-02/videos/penguindemo.mp4
Generated "/home/mackenzie/DEEPLABCUT/DLCdev-DLC2.2/penguins5-teamDLC-2020-03-02/config.yaml"

A new project with name penguins5-teamDLC-2020-03-02 is created at /home/mackenzie/DEEPLABCUT/DLCdev-DLC2.2 and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once you have changed the configuration file, use the function 'extract_frames' to select frames for labeling.
. [OPTIONAL] Use the function 'add_new_videos' to add new videos to your project

'/home/mackenzie/DEEPLABCUT/DLCdev-DLC2.2/penguins5-teamDLC-2020-03-02/config.yaml'

# CRUCIAL: go edit the config.yaml file that was created! 

You will have a config.yaml file in your project folder. 
Crucially, please set:

- **individuals**
        #set the number of animals, i.e. "indiv1, indiv2, indiv3" up to the maximum you expect to see!
- **uniquebodyparts** 
        #this are points that are not tied to an individual, i.e. something like "cornerofabox" or "LED"
- **multianimalbodyparts** 
        #this is the real bodypart per individual list (as in "normal" DLC)
- **skeleton** 
         #this is now used during training, so please connect the bodyparts in a graph

Plus, edit the number of frames to extract per video, etc. 

# Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) ('uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [ ]:
#AUTOMATIC:
deeplabcut.extract_frames(path_config_file) 

# Label Frames

In [ ]:
deeplabcut.label_frames(path_config_file)

# Check Labels, by bodypart:

In [ ]:
deeplabcut.check_labels(path_config_file, visualizeindividuals=False)

# Check Labels, by individual:

You can run this function "again" with the flag set to true and it saves both sets of images.

In [ ]:
deeplabcut.check_labels(path_config_file, visualizeindividuals=True)

# Create maDLC training dataset:

here you have several options, and some new defaults. Firstly, this will use ResNet-50 with imgaug and a new ADAM optimizer with batch training (if your frames are the same size). 

In [ ]:
deeplabcut.create_multianimaltraining_dataset(path_config_file)

## Start training the multi-task DNN + Feature Detectors:
This function trains the network for a specific shuffle of the training dataset. 

Training can be stopped at any time w/CNTRL+C. Note that the weights are only stored every 'save_iters' steps. For this demo the it is advisable to store & display the progress very often (i.e. display every 20, save every 100). In practice this is inefficient (in reality, you will train until ~100K, so we save every 15K).


In [ ]:
deeplabcut.train_network(path_config_file, shuffle=1, saveiters=1000, displayiters=100)
#notice the variables "saveiters" and "dsiplayiters" that can be set in the function

# --------------ONLY UNTIL HERE WORKS

## Evaluate the trained network

This function evaluates a trained model for a specific shuffle/shuffles at a particular training state (snapshot) or on all the states. The network is evaluated on the data set (images) and stores the results as .csv file in a subdirectory under **evaluation-results**.

You can change various parameters in the ```config.yaml``` file of this project. For the evaluation one can change pcutoff. This cutoff also influences how likely estimated postions need to be so that they are shown in the plots.

In [ ]:
deeplabcut.evaluate_network(path_config_file,plotting=True)

## Analyzing videos
This function extracts the pose based on a trained network from videos. The user can choose the trained network - by default the most recent snapshot is used to analyse the videos. However, the user can also specify the snapshot index for the variable **snapshotindex** in the **config.yaml** file).

The results are stored in hd5 file in the same directory, where the video resides. The pose array (pose vs. frame index) can also be exported as csv file (set flag to...). 

In [ ]:
# Set the video path:
#The video can be the one you trained with and new videos that look similar, i.e. same experiments, etc.
# You can add individual videos, OR just a folder - it will skip videos that are already analyzed once.

#i.e you can run 'reachingvideo1' and/or 'MovieS2_Perturbation_noLaser_compressed'

videofile_path = os.path.join(os.getcwd(),'Reaching-Mackenzie-2018-08-30/videos/reachingvideo1.avi')                   

In [ ]:
print("Start Analyzing the video!")
deeplabcut.analyze_videos(path_config_file,[videofile_path])
# this video takes ~ 8 min to analyze with a CPU

*NOTE: Yes, this is slow on a CPU (a GPU is MUCH faster)... see https://www.biorxiv.org/content/early/2018/10/30/457242 if you are interested!*

## Create labeled video

This function is for the visualization purpose and can be used to create a video in .mp4 format with the predicted labels. This video is saved in the same directory, where the (unlabeled) video resides. 

Various parameters can be set with regard to the colormap and the dotsize (matplotlib is used in the backend). See the config.yaml file for how to set these.

In [ ]:
deeplabcut.create_labeled_video(path_config_file,[videofile_path], draw_skeleton=True)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color. The underlying functions can easily be customized.

In [ ]:
%matplotlib notebook
deeplabcut.plot_trajectories(path_config_file,[videofile_path],showfigures=True)

#These plots can are interactive and can be customized (see https://matplotlib.org/)

## Extract outlier frames, where the predictions are off.

This is optional step allows to add more training data when the evaluation results are poor. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [ ]:
# Note, if you have questions on parameters, remember "?" gives you answers:
deeplabcut.extract_outlier_frames?

In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,videofile_path,outlieralgorithm='uncertain',p_bound=.2)

The user can run this iteratively, and (even) extract additional frames from the same video.

## Manually correct labels

This step allows the user to correct the labels in the extracted frames. Navigate to the folder with the videos and use the GUI as described in the protocol to update the labels.

In [ ]:
#GUI pops up! 
#sometimes you need to restart the kernel for the GUI to launch.
%gui wx
deeplabcut.refine_labels(path_config_file)

In [ ]:
# Now merge datasets (once you refined all frames)
deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset, check it and train...

Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [ ]:
#Perhaps plot the labels to see how how all the frames are annotated (including the refined ones)
deeplabcut.check_labels(path_config_file)
# if they are off, you can load them in the labeling_gui to adjust!

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

Now one can train the network again... (with the expanded data set)

In [ ]:
deeplabcut.train_network(path_config_file)